In [1]:
import trimesh
from trimesh import transformations
import numpy as np

sipm_stls = !ls ../data/stl/sipm_tiles/*.STL

stls = [
    "../data/stl/short_tpc_assembly_modulo_anode_sipm.STL",
    "../data/stl/simplified_anode_substrate.STL",
    "../data/stl/simplified_anode.STL",
    "../data/stl/sipm_ceramic_board.STL",
] + sipm_stls

In [2]:
def get_rotmat(axis, rot):
    if axis == "x":
        direction = [1, 0, 0]
    elif axis == "y":
        direction = [0, 1, 0]
    elif axis == "z":
        direction = [0, 0, 1]
    return transformations.rotation_matrix(rot, direction, [0,0,0])

"""
    mesh.apply_translation(translate)'t a great way to find the proper offsets and rotation matrixes
for each STL file. I found these by manually loading the STLs into Blender and
finding the proper offsets and rotations. The offsets are in millimeters and the
rotations are in radians. The rotations are all around the origin of the STL file.
"""

rotmats = [
    np.eye(4),                              # ShortTPC_Full_assembly_noceramicgoldorsipm_plugged
    get_rotmat("y", 112 * np.pi / 180),     # simplified anode substrate
    get_rotmat("y", 112 * np.pi / 180),     # simplified anode
    np.eye(4),                              # SiPM_ceramic_board
] + [get_rotmat("y", 0) for _ in sipm_stls] # SiPMs

translates = [
    [0, 0, 0],                              # ShortTPC_Full_assembly_noceramicgoldorsipm_plugged
    [-2372.5, 643.41, 6452.3],              # simplified anode substrate
    [-1979.5, 590.41, 5542.3],              # simplified anode
    [-3410.3, -6253, -3415.3],              # SiPM_ceramic_board
] + [[-3410.3, -6251.9, -3415.3] for _ in sipm_stls] # SiPMs
translates = np.array(translates) / 100  # convert to cm

def apply_transform(mesh, rotmat, translate):
    mesh.apply_transform(rotmat)
    mesh.apply_translation(translate)
    return mesh

In [3]:
from functools import reduce
import operator

meshes = {}

colors = [
    [200, 200, 200, 100],
    [254, 254, 254, 255],
    [212, 175, 55, 255],
    [254, 254, 254, 255],
] + [[212, 175, 55, 255] for _ in sipm_stls]

for i, (stl, rotmat, translate) in enumerate(zip(stls, rotmats, translates)):
    mesh = trimesh.load_mesh(stl)
    name = stl.split("/")[-1].split(".")[0]

    mesh = apply_transform(mesh, rotmat, translate)
    mesh.visual.face_colors = colors[i]
    meshes[name] = mesh

merged_mesh = reduce(operator.add, meshes.values())z

In [4]:
xavg = (merged_mesh.vertices[:,0].min() + merged_mesh.vertices[:,0].max()) / 2
zavg = (merged_mesh.vertices[:,2].min() + merged_mesh.vertices[:,2].max()) / 2

sipm_y = meshes['SiPM_0'].vertices[:,1].mean()
center_y = ((sipm_y-40) + (sipm_y + 83)) / 2
center = np.array([xavg, center_y, zavg])

radius = 254/2
height = (83+40) / 2

bounds = [
    [xavg-radius, center_y-height, zavg-radius],
    [xavg+radius, center_y+height, zavg+radius]
]

x = np.arange(bounds[0][0], bounds[1][0], 5)
y = np.arange(bounds[0][1], bounds[1][1], 5)
z = np.arange(bounds[0][2], bounds[1][2], 5)
xv, yv, zv = np.meshgrid(x, y, z)

points = np.array([xv.flatten(), yv.flatten(), zv.flatten()]).T

print(points.shape)

(65025, 3)


In [12]:
meshes['short_tpc_assembly_modulo_anode_sipm'].is_volume

False

In [5]:
from scipy.spatial import KDTree

active_volume_mesh = merged_mesh
volume_tree = KDTree(active_volume_mesh.triangles_center)

def is_point_valid(point):
    return volume_tree.query(point)[1]

In [6]:
from tqdm import trange

def find_closest_triangles(i, batch_size, points, tri_centers):
    batch_points = points[i:i+batch_size]
    closest_triangles_batch = np.argmin(np.linalg.norm(tri_centers[:, None] - batch_points[None], axis=-1), axis=0)
    return closest_triangles_batch

# Concatenate the results from all batches
closest_triangles = volume_tree.query(points)[-1]

In [7]:
tri_centers = active_volume_mesh.triangles_center

# Find triangle normal
closest_normals = merged_mesh.face_normals[closest_triangles]

# Find unit vector from point to triangle
closest_vectors = tri_centers[closest_triangles]-points
closest_vectors /= np.linalg.norm(closest_vectors, axis=1)[:, None]

# Find dot product between normal and vector
dot_products = np.sum(closest_normals * closest_vectors, axis=1)

inside = dot_products > 0

In [8]:
# create trimesh lines between closest points and points
closest_points = tri_centers[closest_triangles]
lines = np.array([points, closest_points]).transpose(1, 0, 2)

In [9]:

def render_track(points, sz=0.1, inside=True):
    origin = points[:-1]
    extent = points[1:]-points[:-1]
    perp1 = np.cross(origin,extent)
    perp1 = np.inner(sz/2.0/np.linalg.norm(perp1,axis=1),perp1.T).T
    perp2 = np.cross(perp1,extent)
    perp2 = np.inner(sz/2.0/np.linalg.norm(perp2,axis=1),perp2.T).T
    verts = [perp1+perp2,-perp1+perp2,perp1-perp2,-perp1-perp2]
    bot = [vert+origin for vert in verts]
    top = [vert+origin+extent for vert in verts]
    vertices = [origin,origin+extent,bot[0],top[0],bot[1],top[1],bot[2],top[2],bot[3],top[3]]
    vertices = np.transpose(np.asarray(vertices,np.float32),(1,0,2))
    triangles = np.asarray([[1, 3, 5], [1, 5, 7], [1, 7, 9], [1, 9, 3], [3, 2, 4], [5, 4, 6], [7, 6, 8], [9, 8, 2], [2, 0, 0], [4, 0, 0], [6, 0, 0], [8, 0, 0],
                            [1, 5, 1], [1, 7, 1], [1, 9, 1], [1, 3, 1], [3, 4, 5], [5, 6, 7], [7, 8, 9], [9, 2, 3], [2, 0, 4], [4, 0, 6], [6, 0, 8], [8, 0, 2]],
                                dtype=np.int32)

    # mesh = trimesh.Trimesh(vertices[0], triangles)
    # mesh.visual.face_colors = [[255, 0, 0, 255]]*len(mesh.faces) if inside else [[0, 255, 0, 255]]*len(mesh.faces)
    return vertices[0], triangles

In [10]:
import tqdm

cylinders = [render_track(line, 0.1, i) for line,i in tqdm.tqdm(zip(lines,inside), total=len(lines))]
# cylinders = reduce(operator.add, cylinders)


100%|██████████| 65025/65025 [00:13<00:00, 4765.04it/s]


In [19]:
vertices = np.concatenate([c[0] for c in cylinders])
triangles = np.concatenate([c[1]+10*i for i,c in enumerate(cylinders)])
colors = [[[[255, 0, 0, 255]] * 24 if inside[i] else [[0, 255, 0, 255]] * 24 for i,c in enumerate(cylinders)]]

In [24]:
colors = np.concatenate(colors).ravel().reshape(-1, 4)

In [25]:
m = trimesh.Trimesh(vertices, triangles)
m.visual.face_colors = colors

In [ ]:
(m).show()

In [74]:
voxelized_mesh = merged_mesh.voxelized(5)

In [75]:
filled = voxelized_mesh.copy().fill(method="holes")

In [106]:
import tqdm

fp = filled.points
vp = voxelized_mesh.points

pointdict = dict()
for p in vp:
    pointdict[(p[0], p[1], p[2])] = True

all_pts = [p for p in fp if (p[0], p[1], p[2]) not in pointdict]
all_pts = np.array(all_pts)

colors = [[255,0,0,255] if (p[0], p[1], p[2]) not in pointdict else [0,255,0,255] for p in fp ]

In [115]:
voxelized_mesh.colors = [200, 200, 200, 100]

In [ ]:
pc = trimesh.PointCloud(all_pts, colors=[255,0,0,255])
trimesh.Scene([merged_mesh, pc]).show()

In [48]:
interior_voxels = np.where(voxelized_mesh.matrix)

inverted_voxel_grid = np.zeros_like(voxelized_mesh.matrix, dtype=bool)
inverted_voxel_grid[interior_voxels] = False  # Set interior voxels to False (outside)
inverted_voxel_grid[~voxelized_mesh.matrix] = True  # Set exterior voxels to True (inside)

In [50]:
inside_voxel_grid = trimesh.voxel.VoxelGrid(inverted_voxel_grid, voxelized_mesh.origin, voxelized_mesh.scale)

AttributeError: 'VoxelGrid' object has no attribute 'origin'

In [143]:
stripped_assembly = trimesh.load_mesh('../data/stl/shorttpc_stripped_assembly_plugged.stl')
stripped_assembly.apply_translation([-214.2051239013672, -102.32000297307968, -214.87561798095703])

stripped_assembly += meshes['simplified_anode_substrate'] + meshes['simplified_anode']

In [153]:
voxels = stripped_assembly.voxelized(5)

filled = voxels.copy().fill(method='holes')

fp = filled.points
vp = voxels.points

pointdict = dict()
for p in vp:
    pointdict[(p[0], p[1], p[2])] = True

all_pts = [p for p in fp if (p[0], p[1], p[2]) not in pointdict]
all_pts = np.array(all_pts)

colors = [[255,0,0,255] if (p[0], p[1], p[2]) not in pointdict else [0,255,0,255] for p in fp ]

In [ ]:
pc = trimesh.PointCloud(all_pts, colors=[255,0,0,255])
trimesh.Scene([merged_mesh, pc]).show()